In [56]:
import sparknlp
spark = sparknlp.start() 
# sparknlp.start(gpu=True) >> for training on GPUfrom sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline
import pandas as pd
print("Spark NLP version", sparknlp.version())
print("Apache Spark version:", spark.version)

Spark NLP version 4.2.2
Apache Spark version: 3.3.1


In [57]:
#! wget https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Public/data/news_category_train.csv

In [58]:
#! wget https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Public/data/news_category_test.csv

In [59]:
trainDataset = spark.read \
      .option("header", True) \
      .csv("text_train.csv")
trainDataset.show(10, truncate=50)

+--------------------------------+--------+
|                     description|category|
+--------------------------------+--------+
|           i want to make coffee|      DA|
|                      where am i|      LR|
|              what can i do here|      AR|
|                    prepare meal|      DA|
|                use refrigerator|      DA|
|                         use fan|      DA|
|                        use oven|      DA|
|                       use stove|      DA|
|     i would like to wash sheets|      DA|
|i would like to watch television|      DA|
+--------------------------------+--------+
only showing top 10 rows



In [60]:
testDataset = spark.read \
      .option("header", True) \
      .csv("text_test.csv")
testDataset.show(10, truncate=50)

+--------------------------------+--------+
|                     description|category|
+--------------------------------+--------+
|           i want to make coffee|      DA|
|                      where am i|      LR|
|              what can i do here|      AR|
|                    prepare meal|      DA|
|                use refrigerator|      DA|
|                         use fan|      DA|
|                        use oven|      DA|
|                       use stove|      DA|
|     i would like to wash sheets|      DA|
|i would like to watch television|      DA|
+--------------------------------+--------+
only showing top 10 rows



In [61]:
from pyspark.sql.functions import col
trainDataset.groupBy("category") \
    .count() \
    .orderBy(col("count").desc()) \
    .show()

+--------+-----+
|category|count|
+--------+-----+
|      VC|   46|
|      DA|   29|
|      CC|   26|
|      SD|   23|
|      LR|   21|
|      AR|   20|
|      SN|   20|
|     NAV|   18|
|      NA|    1|
+--------+-----+



In [62]:
testDataset = spark.read \
      .option("header", True) \
      .csv("text_test.csv")
testDataset.groupBy("category") \
    .count() \
    .orderBy(col("count").desc()) \
    .show()

+--------+-----+
|category|count|
+--------+-----+
|      VC|   46|
|      DA|   29|
|      CC|   26|
|      SD|   23|
|      LR|   21|
|      AR|   20|
|      SN|   20|
|     NAV|   18|
|      NA|    1|
+--------+-----+



In [63]:
# actual content is inside description column
document = DocumentAssembler()\
    .setInputCol("description")\
    .setOutputCol("document")
    
# we can also use sentence detector here 
# if we want to train on and get predictions for each sentence# downloading pretrained embeddings
use = BertSentenceEmbeddings.pretrained('sent_small_bert_L8_512')\
    .setInputCols(["document"])\
    .setOutputCol("sentence_embeddings")
# the classes/labels/categories are in category column
classsifierdl = ClassifierDLApproach()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("class")\
    .setLabelColumn("category")\
    .setMaxEpochs(500)\
    .setBatchSize(32)\
    .setRandomSeed(0)\
    .setEnableOutputLogs(True)\
    .setOutputLogsPath('classifer_logs')\

use_clf_pipeline = Pipeline(
    stages = [
        document,
        use,
        classsifierdl
    ])

use_pipelineModel = use_clf_pipeline.fit(trainDataset)


sent_small_bert_L8_512 download started this may take some time.
Approximate size to download 149.1 MB
[OK!]


2023-03-02 12:26:59.423623: I external/org_tensorflow/tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/2fc7d368c66b_classifier_dl10228576928999606746
2023-03-02 12:26:59.482195: I external/org_tensorflow/tensorflow/cc/saved_model/reader.cc:107] Reading meta graph with tags { serve }
2023-03-02 12:26:59.482222: I external/org_tensorflow/tensorflow/cc/saved_model/reader.cc:148] Reading SavedModel debug info (if present) from: /tmp/2fc7d368c66b_classifier_dl10228576928999606746
2023-03-02 12:26:59.774872: I external/org_tensorflow/tensorflow/cc/saved_model/loader.cc:228] Restoring SavedModel bundle.
2023-03-02 12:27:00.202004: I external/org_tensorflow/tensorflow/cc/saved_model/loader.cc:212] Running initialization op on SavedModel bundle at path: /tmp/2fc7d368c66b_classifier_dl10228576928999606746
2023-03-02 12:27:00.312679: I external/org_tensorflow/tensorflow/cc/saved_model/loader.cc:301] SavedModel load for tags { serve }; Status: success: OK. Took 889067 microsec

Training started - epochs: 500 - learning_rate: 0.005 - batch_size: 32 - training_examples: 204 - classes: 9
Epoch 1/500 - 0.19s - loss: 16.323198 - acc: 0.10763889 - batches: 7
Epoch 2/500 - 0.02s - loss: 14.5652075 - acc: 0.21354167 - batches: 7
Epoch 3/500 - 0.02s - loss: 14.755829 - acc: 0.27083334 - batches: 7
Epoch 4/500 - 0.02s - loss: 14.402666 - acc: 0.27083334 - batches: 7
Epoch 5/500 - 0.02s - loss: 14.10333 - acc: 0.31770834 - batches: 7
Epoch 6/500 - 0.02s - loss: 13.936717 - acc: 0.43923613 - batches: 7
Epoch 7/500 - 0.02s - loss: 13.93435 - acc: 0.5503472 - batches: 7
Epoch 8/500 - 0.02s - loss: 13.999413 - acc: 0.5763889 - batches: 7
Epoch 9/500 - 0.02s - loss: 13.941489 - acc: 0.5868055 - batches: 7
Epoch 10/500 - 0.02s - loss: 13.557045 - acc: 0.6614583 - batches: 7
Epoch 11/500 - 0.02s - loss: 13.206701 - acc: 0.7743056 - batches: 7
Epoch 12/500 - 0.02s - loss: 12.90479 - acc: 0.9114583 - batches: 7
Epoch 13/500 - 0.02s - loss: 12.5584755 - acc: 0.9322917 - batches: 

In [64]:
from sklearn.metrics import *
df = use_pipelineModel.transform(testDataset).select("category","description","class").toPandas()
df['result']=df['class'].apply(lambda x: x[0])


In [65]:
use_pipelineModel.stages[2].write().overwrite().save('Text_Classification_Bert')

In [66]:
#predictions =  use_pipelineModel.transform(testDataset)
#predictions.select("category", "text", "class.result").show(5, truncate=30)

In [67]:
# actual content is inside description column
document = DocumentAssembler()\
    .setInputCol("description")\
    .setOutputCol("document")
    
# we can also use sentence detector here 
# if we want to train on and get predictions for each sentence# downloading pretrained embeddings
use = BertSentenceEmbeddings.pretrained('sent_small_bert_L8_512')\
    .setInputCols(["document"])\
    .setOutputCol("sentence_embeddings")
# the classes/labels/categories are in category column
loaded_ner_model = ClassifierDLModel.load("Text_Classification_Bert")\
  .setInputCols(["sentence_embeddings"])\
  .setOutputCol("class")

use_clf_pipeline = Pipeline(
    stages = [
        document,
        use,
        loaded_ner_model
    ])

#use_pipelineModel = use_clf_pipeline.fit(trainDataset)

sent_small_bert_L8_512 download started this may take some time.
Approximate size to download 149.1 MB
[OK!]


In [68]:
from sparknlp.base import LightPipeline
#clf_pipelineModel = use_clf_pipeline.fit(trainDataset)

text = "navigate to kitchen"
prediction_data = spark.createDataFrame([[text]]).toDF("text")
prediction_model = use_clf_pipeline.fit(prediction_data)
light = LightPipeline(prediction_model)
results = light.annotate(text)
print(results)
print(results['class'])

{'document': ['navigate to kitchen'], 'sentence_embeddings': ['navigate to kitchen'], 'class': ['NAV']}
['NAV']


In [69]:
text = input("Enter Testing Text\n")
while(text != "exit"):
    results = light.annotate(text)
    print(results['class'])
    text = input("Enter New Text\n")

['SD']
['SD']
['CC']
['NAV']
['LR']
['DA']
['AR']
['NAV']
['NAV']
